# Begin of ADM 3

In [6]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [21]:
def get_info(html_file):

    soup = bs(html_file)

    place_info = {}


    placeName = soup.find('h1', {'class':'DDPage__header-title'})
    if placeName is not None :
        place_info['placeName'] = placeName.text
    else :
        place_info['placeName'] = ''

    placeTags = soup.find('div', {'class':'DDPage__header-place-location'})
    if placeTags is not None :
        place_info['placeTags'] = placeTags.text
    else :
        place_info['placeTags'] = ''

    if soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'}) is not None and soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}) is not None :
        place_info['numPeopleVisited'] = int(soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}).text)
    else :
        place_info['numPeopleVisited'] = ''

    if soup.find('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'}) is not None and soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}) is not None :
        place_info['numPeopleWant'] = int(soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}).text)
    else :
        place_info['numPeopleWant'] = ''

    placeDesc = soup.find('div', {'id':'place-body'})
    if placeDesc is not None :
        place_info['placeDesc'] = placeDesc.text.replace('\n','')
    else :
        place_info['placeDesc'] = ''
    
    placeShortDesc = soup.find('h3', {'class': 'DDPage__header-dek'})
    if placeShortDesc is not None :
        place_info['placeShortDesc'] = placeShortDesc.text.replace('\n','')
    else :
        place_info['placeShortDesc'] = ''

    placeNearby = set([match.text for match in soup.find_all('div', {'class': 'DDPageSiderailRecirc__item-title'})])
    if len(placeNearby) != 0 :
        place_info['placeNearby'] = ','.join(list(placeNearby))
    else :
        place_info['placeNearby'] = ''

    match_address = soup.find_all("address", {"class":"DDPageSiderail__address"})[0]
    if len(match_address) != 0 :
        placeAddress = match_address.find('div').contents[0:5:2]
        place_info['placeAddress'] = "".join(placeAddress).replace('\n','')
    else :
        place_info['placeAddress'] = ''

    match = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"}).get_text().split()
    if len(match) != 0 :
        place_info['placeAlt'] = float(match[0][:-1])
        place_info['placeLong'] = float(match[1])
    else :
        place_info['placeAlt'] = ''
        place_info['placeLong'] = ''

    contributors = []
    for contributor in soup.find_all('div', {'class':'DDPContributors'})[0].find_all('a', {'class':'DDPContributorsList__contributor'}) :
        if contributor.find('span') is not None :
            contributors.append(contributor.find('span').text)
        else :
            contributors.append(contributor.text)
    if len(contributors) != 0 :
        place_info['placeEditors'] = ','.join(list(set(contributors)))
    else :
        place_info['placeEditors'] = ''

    placePubDate = soup.find("div", {"class":"DDPContributor__name"})
    if placePubDate is not None :
        place_info['placePubDate'] = datetime.strptime(placePubDate.text, '%B %d, %Y')
    else :
        place_info['placePubDate'] = ''

    placeRelatedLists = []
    placeRelatedPlaces = []
    for container in soup.find_all('div', {'class' : 'full-width-container CardRecircSection'}) :
        title = container.find('div', {'class':'CardRecircSection__title'}).text
        if 'Appears' in title :
            for container_related in container.find_all('h3', {'class':'Card__heading --content-card-v2-title js-title-content'}) :
                placeRelatedLists.append(container_related.text[1:-1])
        elif 'Related Places' in title :
            for container_related in container.find_all('h3', {'class':'Card__heading --content-card-v2-title js-title-content'}) :
                placeRelatedPlaces.append(container_related.text[1:-1])
    if len(placeRelatedLists) != 0 :
        place_info['placeRelatedLists'] = ','.join(placeRelatedLists)
    else :
        place_info['placeRelatedLists'] = ''
    if len(placeRelatedPlaces) != 0 :
        place_info['placeRelatedPlaces'] = ','.join(placeRelatedPlaces)
    else :
        place_info['placeRelatedPlaces'] = ''

    if len(soup.find_all("link", {'rel':'canonical'})) != 0 :
        place_info['placeURL'] = soup.find_all("link", {'rel':'canonical'})[0]['href']   
    else :
        place_info['placeURL'] = ''  
        
    return place_info


In [17]:
df = pd.DataFrame(columns=['placeName','placeTags','numPeopleVisited','numPeopleWant','placeDesc','placeShortDesc',
                           'placeNearby','placeAddress','placeAlt','placeLong','placeEditors','placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL']) 


df
df.append(get_info(result.text), ignore_index=True)
